# 1. INSTALL 

## 1.A conda env & Jupyter

```
conda create --name jwst_nircam python=3.9
conda activate jwst_nircam
conda install jupyter 
```

## 1.B jupyter and grizli install
```
pip install "grizli[jwst]"
pip install pyregion
pip install git+https://github.com/gbrammer/dust_attenuation.git
pip install git+https://github.com/gbrammer/dust_extinction.git 
```

## 1.C put these into the ~/.bashrc or equivalent
```
export GRIZLI="${HOME}/grizli" # dir for config files, here or anywhere else 
export iref="${GRIZLI}/iref/"  # for WFC3 calibration files
export jref="${GRIZLI}/jref/"  # for ACS calibration files
mkdir $GRIZLI
mkdir $GRIZLI/CONF      # needed for grism configuration files
mkdir $GRIZLI/templates # for redshift fits
mkdir $iref
mkdir $jref
```

In [ ]:
# Grizli and requirements, e.g., on google collab
import importlib
import sys
import os

try:
    import grizli
except:
    ! pip install "grizli[jwst]"
    ! pip install pyregion
    ! pip install git+https://github.com/gbrammer/dust_attenuation.git
    ! pip install git+https://github.com/gbrammer/dust_extinction.git 

## Module versions
print(sys.version + '\n')
for module in ['grizli', 'tristars', 'mastquery', 'sep',
               'numpy', 'scipy', 'astropy', 'astroquery', 
               'shapely', 'drizzlepac']:
    try:
        mod = importlib.import_module(module)
        print('{0:>20} : {1}'.format(module, mod.__version__))
    except ImportError:
        print('{0:>20} : {1}'.format(module, '*failed*'))

In [4]:
try:
    import jwst
except ImportError:
    ! pip install jwst

if os.getenv('CRDS_PATH') is None:
    cache_dir = os.path.join(os.getcwd(), 'crds_cache')
    print('Set CRDS_PATH=',cache_dir)
    os.environ['CRDS_PATH'] = cache_dir
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)
else:
    print("CRDS_PATH={",os.getenv('CRDS_PATH'),'}')

if os.getenv('CRDS_SERVER_URL') is None:
    server = 'https://jwst-crds.stsci.edu'
    print('Set CRDS_SERVER_URL={',server,'}')
    os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'
else:
    print("CRDS_SERVER_URL={",os.getenv('CRDS_SERVER_URL'),"}")

CRDS_PATH={ /Users/ivo/Astro/PROJECTS/JWST/crds_cache }
CRDS_SERVER_URL={ https://jwst-crds.stsci.edu }


# 2. Download observations from MAST

In [32]:
from astroquery.mast import Observations
from astropy.table import unique, vstack, Table

In [33]:
# get F277W band data from UNCOVER program 2561
obs = Observations.query_criteria(obs_collection = 'JWST', proposal_id = '2561', 
                                  instrument_name = 'Nircam', filters = 'F277W')

In [34]:
# unpack obs list into a Table with each entry corresponding to a unique file
t = [Observations.get_product_list(o) for o in obs]
files = unique(vstack(t), keys='productFilename')
files[0]

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str9,str4,str5,str36,str64,str1,str69,str9,str28,str11,str1,str7,str5,str4,str51,int64,str9,str6,int64
100379303,JWST,image,jw02561001004_02101_00006_nrcblong,source/target (L3) : association generator,S,mast:JWST/product/jw02561-o001_20221118t004454_image2_00345_asn.json,INFO,--,ASN,--,CALJWST,1.8.2,2561,jw02561-o001_20221118t004454_image2_00345_asn.json,1795,100379652,PUBLIC,2


## 2.A First look at "level 3" STScI pipeline reduced coadded images 

In [36]:
# only select "level3" reduced rectified 2D image (i2d.fits)
lvl3 = (files['calib_level'] == 3)
files_lvl3 = vstack( [f for f in files[lvl3] if 'i2d.fits' in f['dataURI']] )
files_lvl3

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str9,str4,str5,str36,str64,str1,str69,str9,str28,str11,str1,str7,str5,str4,str51,int64,str9,str6,int64
100379652,JWST,image,jw02561-o001_t003_nircam_clear-f277w,exposure/target (L2b/L3): rectified 2D image,D,mast:JWST/product/jw02561-o001_t003_nircam_clear-f277w_i2d.fits,SCIENCE,Minimum Recommended Products,I2D,--,CALJWST,1.8.2,2561,jw02561-o001_t003_nircam_clear-f277w_i2d.fits,823775040,100379652,PUBLIC,3
101094748,JWST,image,jw02561-o003_t006_nircam_clear-f277w,exposure/target (L2b/L3): rectified 2D image,D,mast:JWST/product/jw02561-o003_t006_nircam_clear-f277w_i2d.fits,SCIENCE,Minimum Recommended Products,I2D,--,CALJWST,1.8.2,2561,jw02561-o003_t006_nircam_clear-f277w_i2d.fits,342927360,101094748,PUBLIC,3


In [37]:
# download only the first of the two files  
manifest = Observations.download_products(files_lvl3[0])

# add curl flag to get a download script
# manifest = Observations.download_products(files_lvl3, curl_flag=True)

In [23]:
! find . 
# inspect the pipeline reduced files in ds9

.
./abell2744-preimg-02561-001-041.0-nrcb5-f277w-clear_ps1.radec
./jw02561001002_02101_00003_nrcalong_cal.fits
./abell2744-preimg-02561-001-041.0-nrca5-f277w-clear.cat.radec
./jw02561001002_02101_00003_nrcblong_cal_onef_axis1.png
./abell2744-preimg-02561-001-041.0-nrcb5-f277w-clear_ps1.reg
./jw02561001002_02101_00001_nrcalong_cal.wcslog.fits
./uncover_visits.yaml
./jw02561001002_02101_00002_nrcalong_cal.fits
./abell2744-preimg-02561-001-041.0-nrca5-f277w-clear_ps1.radec
./abell2744-preimg-02561-001-041.0-nrca5-f277w-clear_wcs.fits
./abell2744-preimg-02561-001-041.0-nrcb5-f277w-clear.cat.fits
./abell2744-preimg-02561-001-041.0-nrcb5-f277w-clear.cat.radec
./jw02561001002_02101_00003_nrcalong_cal_onef_axis1.png
./abell2744-preimg-02561-001-041.0-nrcb5-f277w-clear_drz_sci.fits
./jw02561001002_02101_00001_nrcblong_cal.wcslog.fits
./jw02561001002_02101_00004_nrcalong_cal.wcslog.fits
./abell2744-preimg-02561-001-041.0-nrcb5-f277w-clear.cat.reg
./jw02561001002_02101_00001_nrcalong_cal.fits
./a

## 2.B download level 2b calibrated frames

In [39]:
# download level 2b products (calibrated individual frames ending in _cal.fits)
lvl2 = (files['calib_level'] == 2) & (files['productType'] == 'SCIENCE')
files_lvl2 = vstack( [f for f in files[lvl2] if 'cal.fits' in f['dataURI']] )

files_lvl2[0]

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str9,str4,str5,str36,str64,str1,str69,str9,str28,str11,str1,str7,str5,str4,str51,int64,str9,str6,int64
99946043,JWST,image,jw02561001002_02101_00001_nrcalong,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw02561001002_02101_00001_nrcalong_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.8.2,2561,jw02561001002_02101_00001_nrcalong_cal.fits,117538560,100379652,PUBLIC,2


In [40]:
# download only the first 8 files (4 exposures of each a/b module)  
manifest = Observations.download_products(files_lvl2[0:8])

# Reduction of level 2b cal frames with Grizli

In [98]:
print(os.environ['CRDS_PATH'])
print(os.environ["CRDS_SERVER_URL"])
#os.environ['CRDS_CONTEXT'] = 'jwst_0863.pmap'
#print(os.environ['CRDS_CONTEXT'])

/Users/ivo/Astro/PROJECTS/JWST/crds_cache
https://jwst-crds.stsci.edu


In [28]:
# setup paths and files for grizli
from pathlib import Path
from glob import glob

import os
import grizli
from grizli import utils, prep, jwst_utils, multifit
from grizli.pipeline import auto_script
print('Grizli version: ', grizli.__version__)

# This rootname used throughout
root='uncover'            

# Some directories needed for grizli 
if 'HOME_PATH' not in locals(): HOME_PATH=os.getcwd()
working_dir = 'Prep'   # hardcoded in autoscript grizli... beware !
raw_dir = '../RAW'

Path(working_dir).mkdir(exist_ok=True)
os.chdir(HOME_PATH+'/'+working_dir)
Path(raw_dir).mkdir(exist_ok=True)
os.chdir(HOME_PATH)

# copy cal files into RAW directory
! find . -name "*cal.fits" -exec cp {} RAW \;

Grizli version:  1.7
cp: RAW/jw02561001002_02101_00003_nrcalong_cal.fits and ./RAW/jw02561001002_02101_00003_nrcalong_cal.fits are identical (not copied).
cp: RAW/jw02561001002_02101_00002_nrcalong_cal.fits and ./RAW/jw02561001002_02101_00002_nrcalong_cal.fits are identical (not copied).
cp: RAW/jw02561001002_02101_00001_nrcalong_cal.fits and ./RAW/jw02561001002_02101_00001_nrcalong_cal.fits are identical (not copied).
cp: RAW/jw02561001002_02101_00004_nrcalong_cal.fits and ./RAW/jw02561001002_02101_00004_nrcalong_cal.fits are identical (not copied).
cp: RAW/jw02561001002_02101_00001_nrcblong_cal.fits and ./RAW/jw02561001002_02101_00001_nrcblong_cal.fits are identical (not copied).
cp: RAW/jw02561001002_02101_00004_nrcblong_cal.fits and ./RAW/jw02561001002_02101_00004_nrcblong_cal.fits are identical (not copied).
cp: RAW/jw02561001002_02101_00003_nrcblong_cal.fits and ./RAW/jw02561001002_02101_00003_nrcblong_cal.fits are identical (not copied).
cp: RAW/jw02561001002_02101_00002_nrcblon